Author: ProtonX Team

Website: https://protonx.ai/

## 1. Load dữ liệu

In [ ]:
# Hướng dẫn về bộ dữ liệu này được thực hiện trong Lab Tokenizer
!wget --no-check-certificate https://storage.googleapis.com/learning-datasets/sarcasm.json -O /tmp/sarcasm.json

import json

with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

In [ ]:
# https://storage.googleapis.com/learning-datasets/sarcasm.jsondatastore[0]

## 2. Tách từ

In [ ]:
dataset = []
label_dataset = []

for item in datastore:
    dataset.append(item["headline"])
    label_dataset.append(item["is_sarcastic"])

In [ ]:
dataset[:10], label_dataset[:10]

In [ ]:
import numpy as np

dataset = np.array(dataset)
label_dataset = np.array(label_dataset)

In [ ]:
dataset

Chia dữ liệu

In [ ]:
train_size = 0.8
size = int(len(dataset) * train_size)

train_sentence = dataset[:size]
test_sentence = dataset[size:]

train_label = label_dataset[:size]
test_label = label_dataset[size:]

In [ ]:
len(train_sentence), len(test_sentence)

### 2.1. Train_sequences

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size

In [ ]:
vocab_size = len(train_sentence)
embedding_size = 64
max_length = 25

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentence)

In [ ]:
tokenizer.word_index

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentence)

In [ ]:
train_sentence[1]

In [ ]:
train_sequences[1]

In [ ]:
padded_train_sequences = pad_sequences(train_sequences, maxlen=max_length, truncating="post", padding="post")

In [ ]:
max_length

In [ ]:
padded_train_sequences[2:4]

In [ ]:
padded_train_sequences.shape

### 2.2. Test_sequences

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_sentence)

In [ ]:
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, truncating="post", padding="post")

In [ ]:
padded_test_sequences.shape

Custom Model LSTM

In [ ]:
import tensorflow as tf

Lập trình thuật toán cho một cell LSTM bao gồm các cổng đề điều chỉnh luồng thông tin:
- Cổng kiểm soát đầu vào
- Cổng kiểm soát số lượng dữ liệu giữ lại/quên đi
- Cổng kiểm soát dữ liệu đầu ra

Code dưới đây kế thừa class Layer từ keras để override một số hàm cho mô hình cụ thể của chúng ta.

[Xem thêm](https://www.tensorflow.org/tutorials/customization/custom_layers) cách sử dụng tại.

In [ ]:
class LSTM(tf.keras.layers.Layer):
  def __init__(self, units, inp_shape):
    super(LSTM, self).__init__()
    self.units = units
    self.inp_shape = inp_shape
    self.W = self.add_weight("W", shape=(4, self.units, self.inp_shape))
    self.U = self.add_weight("U", shape=(4, self.units, self.units))


  def call(self, pre_layer, x):
    pre_h, pre_c = tf.unstack(pre_layer)

    # Cổng kiểm soát đầu vào: Input Gate

    i_t = tf.nn.sigmoid(tf.matmul(x, tf.transpose(self.W[0])) + tf.matmul( pre_h, tf.transpose(self.U[0])))

    # Cổng kiểm soát số lượng dữ liệu giữ lại/quên đi: Forget Gate
    f_t = tf.nn.sigmoid(tf.matmul(x, tf.transpose(self.W[1])) + tf.matmul( pre_h, tf.transpose(self.U[1])))

    # Cổng kiểm soát dữ liệu đầu ra: Output Gate
    o_t = tf.nn.sigmoid(tf.matmul(x, tf.transpose(self.W[2])) + tf.matmul( pre_h, tf.transpose(self.U[2])))

    # Đây giống SimpleRNN và được coi là thông tin mới (bộ nhớ mới)

    n_c_t = tf.nn.tanh(tf.matmul(x, tf.transpose(self.W[3])) + tf.matmul( pre_h, tf.transpose(self.U[3])))

    # Kết hợp việc giữ lại thông tin + bổ sung thêm thông tin mới

    c = tf.multiply(f_t, pre_c) + tf.multiply(i_t, n_c_t)

    # Cho phép bao nhiêu thông tin thoát khỏi cell

    h = tf.multiply(o_t, tf.nn.tanh(c))

    return tf.stack([h, c])


## 3. Xây dựng mô hình

Thừa kế lại lớp Model của Keras để xây dựng mô hình riêng cho mình.



Mô hình bao gồm
- Embedding để chuyển câu thành các vector với chiều `embedding_size`
- Sử dụng cell LSTM đã lập trình bên trên
- Sau đó đưa lịch sử `h` của LSTM qua mạng nơ ron đơn giản

Qúa trình training bao gồm
- Đưa câu qua Embedding để lấy các vector
- Đưa `tuần tự` từng từ qua LSTM + (hidden_state và cell_state) lớp phía trước để thu được (hidden_state và cell_state) hiện tại
- Sử dụng hidden_state cuối cùng cho việc dự đoán

In [ ]:
import numpy as np

In [ ]:
class Bidirectional(tf.keras.Model):
  def __init__(self, lstm1, lstm2):
    super(Bidirectional, self).__init__()
    self.lstm1 = lstm1
    self.lstm2 = lstm2

  def call(self):
    # Left to right


    # Right to left
    pass

In [ ]:
class ProtonXRNN(tf.keras.Model):
  def __init__(self, units, embedding_size, vocab_size, input_length):
    super(ProtonXRNN, self).__init__()
    self.input_length = input_length
    self.units = units

    # Embedding để chuyển từ thành vector
    self.embedding = tf.keras.layers.Embedding(
      vocab_size,
      embedding_size,
      input_length = input_length
    )

    # Sử dụng cell LSTM đã lập trình bên trên
    self.lstm = LSTM(units, embedding_size)

    # Sau đó đưa lịch sử h của LSTM qua mạng nơ ron đơn giản
    self.classfication_model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=(units,)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])


  def call(self, sentence):
    """
    Parameters:
    sentence:
      Dạng: Tensor
      Miêu tả: Câu
      Chiều: (batch_size, input_length)
    out:
      Dạng: Tensor
      Miêu tả: Đầu ra của mô hình dự đoán
      Chiều: (batch_size, 1)
    """

    batch_size = tf.shape(sentence)[0]

    # Khởi tạo (hidden_state và context_state)

    pre_layer = tf.stack([
      tf.zeros([batch_size, self.units]),
      tf.zeros([batch_size, self.units])
    ])

    # Đưa câu qua Embedding để lấy các vector
    # embedded_sentence: (batch_size, input_length, embedding_size)
    embedded_sentence = self.embedding(sentence)

    # Đưa tuần tự từng từ qua LSTM + (hidden_state và cell_state)
    # lớp phía trước để thu được (hidden_state và cell_state) hiện tại
    for i in range(self.input_length):
      # : đầu tiên: Lấy batch_size
      # i Vị trí từ
      # : cuối cùng: Lấy embedding.
      # (batch_size, embedding_size)
      word = embedded_sentence[:, i, :]
      pre_layer = self.lstm(pre_layer, word)


    h, _ = tf.unstack(pre_layer)

    # Sử dụng hidden_state cuối cùng cho việc dự đoán
    return self.classfication_model(h)

units = 128
embedding_size = 100
vocab_size = len(tokenizer.index_word) + 1
input_length = max_length

# Khởi tạo đối tượng protonxrnn
protonxrnn = ProtonXRNN(units, embedding_size, vocab_size, input_length)


protonxrnn.compile(
    tf.keras.optimizers.Adam(0.0005) , loss='binary_crossentropy', metrics=['acc']
)

# Tiến hành training

protonxrnn(padded_train_sequences[10:13]).shape

TensorShape([3, 1])

## 4. Tiến hành training

In [ ]:
protonxrnn.fit(padded_train_sequences, train_label, validation_data=(padded_test_sequences, test_label) ,batch_size=32, epochs=10)

Epoch 1/10
668/668 [==============================] - 44s 46ms/step - loss: 0.5625 - acc: 0.6732 - val_loss: 0.3688 - val_acc: 0.8368
Epoch 2/10
668/668 [==============================] - 17s 26ms/step - loss: 0.2638 - acc: 0.8937 - val_loss: 0.3363 - val_acc: 0.8516
Epoch 3/10
668/668 [==============================] - 18s 27ms/step - loss: 0.1471 - acc: 0.9479 - val_loss: 0.4384 - val_acc: 0.8499
Epoch 4/10
668/668 [==============================] - 17s 25ms/step - loss: 0.0877 - acc: 0.9708 - val_loss: 0.4103 - val_acc: 0.8450
Epoch 5/10
668/668 [==============================] - 17s 25ms/step - loss: 0.0571 - acc: 0.9804 - val_loss: 0.6194 - val_acc: 0.8366
Epoch 6/10
668/668 [==============================] - 17s 26ms/step - loss: 0.0376 - acc: 0.9879 - val_loss: 0.8931 - val_acc: 0.8278
Epoch 7/10
668/668 [==============================] - 17s 26ms/step - loss: 0.0243 - acc: 0.9929 - val_loss: 0.6982 - val_acc: 0.8340
Epoch 8/10
668/668 [==============================] - 18s 28ms